In [1]:
import pyspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("yarn")\
        .appName("movie")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar")\
        .config("spark.sql.shuffle.partitions", "7") \
        .config("spark.sql.warehouse.dir", "hdfs:/user/bigdata/repository/database/") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.executor.cores", "4") \
        .config("spark.dynamicAllocation.minExecutors", "1") \
        .config("spark.dynamicAllocation.maxExecutors", "5") \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/08 23:28:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/08 23:28:04 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/eduardoalberto/Library/Python/3.9/lib/python/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/eduardoalberto/Library/Python/3.9/lib/python/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socket.py", line 7

KeyboardInterrupt: 

In [2]:
df = spark.read.option("delimiter", "\t").option("header", True).csv('hdfs:/user/bigdata/repository/input/data_2022*.csv')
df.show(truncate=False)

+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nconst   |primaryName        |birthYear|deathYear|primaryProfession                    |knownForTitles                         |
+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nm0000001|Fred Astaire       |1899     |1987     |soundtrack,actor,miscellaneous       |tt0050419,tt0072308,tt0031983,tt0053137|
|nm0000002|Lauren Bacall      |1924     |2014     |actress,soundtrack                   |tt0037382,tt0038355,tt0117057,tt0071877|
|nm0000003|Brigitte Bardot    |1934     |\N       |actress,soundtrack,music_department  |tt0056404,tt0057345,tt0049189,tt0054452|
|nm0000004|John Belushi       |1949     |1982     |actor,soundtrack,writer              |tt0077975,tt0072562,tt0080455,tt0078723|
|nm0000005|Ingmar Bergman     |1918     |2007     |writer,director,actor                |t

In [ ]:
# df.filter("primaryProfession in ('soundtrack')").show()
df = (df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N'  ")
        .withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
        .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))

        

)
dfs = df.select(F.col("nconst"),
                F.col("primaryName"),
                F.col("birthYear"),
                F.col("deathYear"),
                F.explode(F.col("profession_array")).alias("profession"),
                F.explode(F.col("knownForTitles_array")).alias("Titles"))

                
dfs.show()

#123.627.002
#11.845.112
#357.801


In [12]:
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *

def dp_etl(df: DataFrame) -> DataFrame :

    dfs = (df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N' ")
                .withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
                .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))
                .select(F.col("nconst"),
                        F.col("primaryName"),
                        F.col("birthYear"),
                        F.col("deathYear"),
                        F.explode(F.col("profession_array")).alias("profession"),
                        F.explode(F.col("knownForTitles_array")).alias("Titles"))
    )
    dffull = (dfs.filter("profession <> r'\\N' and Titles <> r'\\N' ")
                 .dropDuplicates(["profession","Titles"]))
    return dffull


df = spark.read.option("delimiter", "\t").option("header", True).csv('hdfs:/user/bigdata/repository/input/data_2022*.csv')
tb = dp_etl(df)

tb.write.mode('overwrite').partitionBy("deathYear").parquet("/user/bigdata/repository/parquet/movie.parquet")

#spark.table("dbdev001.tb_movie").show()
#spark.sql("drop table if exists movie")
#tb.limit(1500).write.mode('overwrite').partitionBy("deathYear").saveAsTable("movie")
# tb.limit(1000).write.mode("overwrite").jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.movie",properties={"user": "root", "password": "mysql"})



In [14]:
# spark.sql("show partitions movie").show()
# spark.sql("desc table extended movie").show()
# spark.table("movie").count()

df02 = spark.read.parquet("/user/bigdata/repository/parquet/movie.parquet")
df02.printSchema()




#3.449.518






#123.627.027
#11.845.112

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- profession: string (nullable = true)
 |-- Titles: string (nullable = true)
 |-- deathYear: string (nullable = true)

